<a href="https://colab.research.google.com/github/scaairesearch/da_demo/blob/main/Gradio_C1_C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.9 MB/s et

In [3]:
import gradio as gr
import os
from PIL import Image
from torchvision import datasets,transforms
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from collections import OrderedDict
import pandas as pd
import io
import base64

In [4]:
# checking the mounted drive and mounting if not done
if not os.path.exists('/content/gdrive'):
  from google.colab import drive
  drive.mount('/content/gdrive')
else:
    print("Google Drive is already mounted.")

Mounted at /content/gdrive


In [7]:
list_c1 = torch.load('/content/gdrive/MyDrive/da_demo/cv/models/26_06/list_mnist_m_non_dann_misclassified_dann_classified.pt')

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        imgs, labels, image_names = self.data[idx]
        return imgs, labels, image_names

dataset_c1 = CustomDataset(list_c1)

In [15]:
# Create a dataloader with the filtered dataset
dataloader_c1 = torch.utils.data.DataLoader(dataset_c1, batch_size=10, shuffle=True)

In [16]:
transform_to_pil  = transforms.ToPILImage()

def get_images():
    images, labels,image_names = next(iter(dataloader_c1))
    pil_images = [transform_to_pil(image) for image in images]
    return pil_images, labels.tolist()



In [28]:
list_c2 = torch.load('/content/gdrive/MyDrive/da_demo/cv/models/26_06/list_mnist_m_non_dann_misclassified_dann_misclassified.pt')
dataset_c2 = CustomDataset(list_c2)
dataloader_c2 = torch.utils.data.DataLoader(dataset_c2, batch_size=10, shuffle=True)
def get_images_2():
    images, labels,image_names = next(iter(dataloader_c2))
    pil_images = [transform_to_pil(image) for image in images]
    return pil_images, labels.tolist()

In [ ]:
# next(iter(dataloader_c1))

In [19]:
def get_device():
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    print("Device Selected:", device)
    return device

device = get_device()


Device Selected: cpu


In [20]:

class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class Network(nn.Module):
    def __init__(self, num_classes = 10):
        super(Network, self).__init__()  # Initialize the parent class

        drop_out_value = 0.1

        #---------------------Feature Extractor Network------------------------#
        self.feature_extractor  = nn.Sequential(
            # Input Block
            nn.Conv2d(3, 16, 3, bias=False),  # In: 3x28x28, Out: 16x26x26, RF: 3x3, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 2
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x26x26, Out: 16x24x24, RF: 5x5, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 3
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x24x24, Out: 16x22x22, RF: 7x7, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Transition Block 1
            nn.MaxPool2d(kernel_size=2, stride=2),  # In: 16x22x22, Out: 16x11x11, RF: 8x8, Stride: 2

            # Conv Block 4
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x11x11, Out: 16x9x9, RF: 12x12, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 5
            nn.Conv2d(16, 32, 3, bias=False),  # In: 16x9x9, Out: 32x7x7, RF: 16x16, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(drop_out_value),

            # Output Block
            nn.Conv2d(32, 64, 1, bias=False),  # In: 32x7x7, Out: 64x7x7, RF: 16x16, Stride: 1

            # Global Average Pooling
            nn.AvgPool2d(7)  # In: 64x7x7, Out: 64x1x1, RF: 16x16, Stride: 7
        )

        #---------------------Class Classifier Network------------------------#
        self.class_classifier = nn.Sequential(nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(64,50),
                                        nn.BatchNorm1d(50), # added batch norm to improve accuracy
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(50,num_classes))

        #---------------------Label Classifier Network------------------------#
        self.domain_classifier = nn.Sequential(nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(64,50),
                                        nn.BatchNorm1d(50), # added batch norm to improve accuracy
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(50,2))
    def forward(self, input_data, alpha = 1.0):
      if input_data.data.shape[1] == 1:
        input_data = input_data.expand(input_data.data.shape[0], 3, img_size, img_size)

      input_data = self.feature_extractor(input_data)

      features = input_data.view(input_data.size(0), -1)  # Flatten the output for fully connected layer

      reverse_features = GradientReversalFn.apply(features, alpha)
      class_output = self.class_classifier(features)
      domain_output = self.domain_classifier(reverse_features)

      return class_output, domain_output, features

In [ ]:
## NON DANN
# Instantiate the model (make sure it has the same architecture)
loaded_model_non_dann = Network()
loaded_model_non_dann = loaded_model_non_dann.to(device)
# Load the saved state dictionary
loaded_model_non_dann.load_state_dict(torch.load('/content/gdrive/MyDrive/da_demo/cv/models/26_06/non_dann_26_06.pt', map_location=device), strict=False)
loaded_model_non_dann.eval()

In [ ]:
##  DANN
# Instantiate the model (make sure it has the same architecture)
loaded_model_dann = Network()
loaded_model_dann = loaded_model_dann.to(device)
# Load the saved state dictionary
loaded_model_dann.load_state_dict(torch.load('/content/gdrive/MyDrive/da_demo/cv/models/26_06/dann_26_06.pt', map_location=device), strict=False)
loaded_model_dann.eval()

In [24]:
img_size = 28 # for mnist
cpu_batch_size = 10
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [25]:
def classify_image(image,
                   select_model
                   ):
  if select_model == "Baseline (Non-DANN)":
    model = loaded_model_non_dann
    test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])
    transformed_image = test_transforms(image)
    image_tensor = transformed_image.to(device).unsqueeze(0)
  if select_model == "DANN":
    model = loaded_model_dann
    target_test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                                       ])
    target_transformed_image = target_test_transforms(image)
    image_tensor = target_transformed_image.to(device).unsqueeze(0)


  logits,_,_ = model(image_tensor)
  output = F.softmax(logits.view(-1), dim = -1) #F.softmax(output.flatten(), dim=-1) #

  confidences = [(class_names[i], float(output[i])) for i in range(len(class_names))]
  confidences.sort(key=lambda x: x[1], reverse=True)
  confidences = OrderedDict(confidences[:3])
  label = torch.argmax(output).item()

  return confidences


In [34]:
def classify_image_both(image
                   ):
  target_test_transforms = transforms.Compose([
                                    transforms.Resize(img_size),
                                    transforms.ToTensor(),# converts to tesnor
                                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                                    ])
  target_transformed_image = target_test_transforms(image)
  image_tensor = target_transformed_image.to(device).unsqueeze(0)

  list_confidences = []
  for model in [loaded_model_non_dann, loaded_model_dann]:
    model.eval()
    logits,_,_ = model(image_tensor)
    output = F.softmax(logits.view(-1), dim = -1)

    confidences = [(class_names[i], float(output[i])) for i in range(len(class_names))]
    confidences.sort(key=lambda x: x[1], reverse=True)
    confidences = OrderedDict(confidences[:3])
    label = torch.argmax(output).item()
    list_confidences.append(confidences)


  return list_confidences[0],list_confidences[1]

In [39]:
with gr.Blocks() as demo:
  with gr.Tab("Case 1: MNIST_M_Non_DANN_Misclassify_DANN_Classify"):
    with gr.Row():
      radio_model = gr.Radio(["Baseline (Non-DANN)", "DANN"],
                               label="Select the model you want to use.",
                               value="Baseline (Non-DANN)",  # Set default value
                               scale=2)
    with gr.Row():
      with gr.Column():
        input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify = gr.Button("Submit to Classify Image", visible = True, size ='sm')

      with gr.Column():
        with gr.Row():
          label_classify = gr.Label(label = "Predicted label", num_top_classes=10, visible = True)

    mnist_m_images,mnist_m_labels = get_images()

    with gr.Row():
      gr.Examples(mnist_m_images,inputs=[input_image_classify], label = "Select an example MNIST-M Image") #working

    with gr.Row():
      gr.Markdown(value = f'MNIST- M Ground Truth Label = {[label for label in mnist_m_labels]}')


    button_classify.click(fn=classify_image,
                          inputs=[input_image_classify,radio_model],
                          outputs=[label_classify])

  with gr.Tab("Case 2: MNIST_M_Both_Misclassify"):
    with gr.Row():
      radio_model = gr.Radio(["Baseline (Non-DANN)", "DANN"],
                               label="Select the model you want to use.",
                               value="Baseline (Non-DANN)",  # Set default value
                               scale=2)
    with gr.Row():
      with gr.Column():
        input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify = gr.Button("Submit to Classify Image", visible = True, size ='sm')

      with gr.Column():
        with gr.Row():
          label_classify = gr.Label(label = "Predicted label", num_top_classes=10, visible = True)

    mnist_m_images,mnist_m_labels = get_images_2()

    with gr.Row():
      gr.Examples(mnist_m_images,inputs=[input_image_classify], label = "Select an example MNIST-M Image") #working

    with gr.Row():
      gr.Markdown(value = f'MNIST- M Ground Truth Label = {[label for label in mnist_m_labels]}')


    button_classify.click(fn=classify_image,
                          inputs=[input_image_classify,radio_model],
                          outputs=[label_classify])

  with gr.Tab("Case 2 - Show both: MNIST_M_Both_Misclassify"):

    with gr.Row():
      with gr.Column():
        input_image_classify_both = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify_both = gr.Button("Submit to Classify Image with Both Models", visible = True, size ='sm')

      with gr.Column():
        with gr.Row():
          label_classify_non_dann = gr.Label(label = "NON DANN Predicted label", num_top_classes=3, visible = True)
        with gr.Row():
          label_classify_dann = gr.Label(label = "DANN Predicted label", num_top_classes=3, visible = True)

    mnist_m_images,mnist_m_labels = get_images_2()

    with gr.Row():
      gr.Examples(mnist_m_images,inputs=[input_image_classify_both], label = "Select an example MNIST-M Image") #working

    with gr.Row():
      gr.Markdown(value = f'MNIST- M Ground Truth Label = {[label for label in mnist_m_labels]}')


    button_classify_both.click(fn=classify_image_both,
                          inputs=[input_image_classify_both],
                          outputs=[label_classify_non_dann,label_classify_dann])


demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e80ef38a8495e62e3f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e80ef38a8495e62e3f.gradio.live
